# **Doublet Panel Method**

In [1]:
include("../src/PanelMethods.jl")
include("../src/FoilParametrization.jl")
using Revise
using .PanelMethods: DoubletPanel2D, Uniform2D, doublet_panels, collocation_point, split_panels, grid_data, aero_coefficients, pressure_coefficient, ×
using .FoilParametrization: read_foil, cosine_foil, kulfan_CST, naca4, linspace
using BenchmarkTools
using Plots
plotly()

Plots.PlotlyBackend()

### Kulfan CST Airfoil

In [9]:
alpha_u = [0.2, 0.3, 0.2, 0.15, 0.2]
alpha_l = [-0.2, -0.1, -0.1, -0.001, -0.02]
alphas = [alpha_u alpha_l]
dzs = (1e-4, 1e-4)
airfoil = kulfan_CST(alphas, dzs, 0.1, 100)
airfoil = naca4((2, 4, 1, 2), 50, sharp_trailing_edge = true)
# airfoil = cosine_foil(read_foil("../airfoil_database/clarky.dat"), n = 40)

99×2 Array{Float64,2}:
 1.0       -1.66165e-17
 0.998983   0.000217318
 0.995934   0.000866504
 0.990867   0.00193932
 0.9838     0.00342226
 0.97476    0.00529687
 0.963783   0.00754014
 0.95091    0.010125
 0.936193   0.0130211
 0.919688   0.0161949
 0.90146    0.0196107
 0.881581   0.0232312
 0.86013    0.0270176
 ⋮         
 0.879865  -0.00890834
 0.899954  -0.00746709
 0.9184    -0.00612897
 0.935126  -0.00490231
 0.950059  -0.0037954
 0.963134  -0.00281629
 0.974296  -0.00197268
 0.983495  -0.00127163
 0.990692  -0.000719378
 0.995856  -0.000321044
 0.998963  -8.04613e-5
 1.0        1.66165e-17

### Setup

In [10]:
uniform = Uniform2D(1.0, 3.0)
panels = doublet_panels(airfoil);

In [11]:
@time cl, kcl, cps = aero_coefficients(panels, uniform)
println("Lift Coefficient: $cl, $kcl")

  0.003518 seconds (11.52 k allocations: 1.208 MiB)
Lift Coefficient: 0.6136202145529182, 0.6145060580248396


### Plotting

Grid parameters

In [12]:
x_domain, y_domain = (-1, 2), (-1, 1)
grid_size = 50
x_dom, y_dom = linspace(x_domain..., grid_size), linspace(y_domain..., grid_size)
grid = x_dom × y_dom

vels, pots = grid_data(panels, grid)

cp = pressure_coefficient.(vels, uniform.mag);

In [13]:
plot(airfoil[:,1], airfoil[:,2], markershape = :circle, aspect_ratio = :equal)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl

In [14]:
upper_panels, lower_panels = split_panels(panels)
plot([ (first ∘ collocation_point)(panel) for panel in upper_panels ], [ panel.cp for panel in upper_panels ], 
     label = "Upper", markershape = :circle)
plot!([ (first ∘ collocation_point)(panel) for panel in lower_panels ], [ panel.cp for panel in lower_panels ], 
      label = "Lower", markershape = :circle, yaxis = (:flip))

<!DOCTYPE html>
 
 
 Plots.jl

In [15]:
p1 = contour(x_dom, y_dom, cp, fill = true)
plot(p1)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), color = "black", aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl